# Test with own multicell data

In [ ]:
%matplotlib inline

# path hack for relative import in jupyter notebook
import os
import sys

# LIBRARY GLOBAL MODS
CELLTYPES = os.path.dirname(os.path.abspath(''))
sys.path.append(CELLTYPES)

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas as pd
import umap
import time
%matplotlib inline

sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

from utils.file_io import RUNS_FOLDER

# TODO 1: bokeh plot with s%d
# TODO 2: visualize s%d, get energy of s%d

In [ ]:
def gen_control_data(total_spins, num_runs):
    X_01 = np.random.randint(2, size=(num_runs, total_spins))
    X = X_01 * 2 - 1
    return X

**Allign UMAP but for Gamma (not step)**

In [ ]:
REDUCER_SEED = 100
REDUCER_COMPONENTS = 2
UMAP_KWARGS = {
    'random_state': REDUCER_SEED,
    'n_components': REDUCER_COMPONENTS,
    'metric': 'euclidean',
    'init': 'spectral',
    'unique': False,
    'n_neighbors': 15,
    'min_dist': 0.1,
    'spread': 1.0,
}

use_01 = True
nn = 4000  # runs with 1000, crash with 5000, 10000 -- try to restrict to more int gammas maybe
kk = 900   # debug: subsample multicell spins to avoid memory issue

In [ ]:
# Step 0) which 'manyruns' dirs to work with
#gamma_list = [0.0, 0.05, 0.1, 0.2, 1.0, 2.0, 20.0]
gamma_list = [0.0, 0.02, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1.0, 2.0, 20.0]

manyruns_dirnames = ['Wmaze15_R5_gamma%.2f_10k_p3_M100' % a for a in gamma_list]
#manyruns_dirnames = ['Wrandom0_gamma%.2f_10k_fixedorder_p3_M4' % a for a in gamma_list]
# manyruns_dirnames = ['Wrandom1_gamma%.2f_10k_fixedorder_ferro' % a for a in gamma_list]
# manyruns_dirnames = ['Wrandom0_gamma%.2f_10k_fixedorder_p3_M100' % a for a in gamma_list]
# manyruns_dirnames = ['Wrandom1_gamma%.2f_10k_fixedorder_p3_M100' % a for a in gamma_list]
# manyruns_dirnames = ['Wrandom0_gamma%.2f_10k_periodic_fixedorderV3_p3_M100' % a for a in
#                     gamma_list]

# gamma_list = [0.0, 0.05, 0.1, 0.2, 1.0, 2.0, 20.0]
# gamma_list = [0.0, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.15, 0.20, 0.4, 0.6, 0.8, 0.9, 1.0, 20.0]
# gamma_list = [0.0, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.15, 0.20, 0.4, 0.6, 0.8, 0.9, 1.0, 20.0]
# gamma_list = [0.0, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.15, 0.20, 0.4, 0.6, 0.8, 1.0, 20.0]
# gamma_list = [0.0, 0.05]  # , 0.06]# , 0.07] # , 0.08, 0.09, 0.10] #, 0.15, 0.20, 0.4, 0.6, 0.8, 0.9, 1.0, 20.0]
# gamma_list = [2.0, 20.0]
# gamma_list = gamma_list[::-1]

manyruns_paths = [RUNS_FOLDER + os.sep + 'multicell_manyruns' + os.sep + dirname
                  for dirname in manyruns_dirnames]

# Step 1) umap (or other dim reduction) kwargs
n_components = 2

X_multi = np.zeros((len(gamma_list), nn, kk), dtype=int)
for j, manyruns_path in enumerate(manyruns_paths):

    gamma_val = gamma_list[j]

    umap_kwargs = UMAP_KWARGS.copy()
    umap_kwargs['n_components'] = n_components  # TODO don't need to spec 'live', can embed later?
    # modify umap settings
    # umap_kwargs['unique'] = True
    # umap_kwargs['n_neighbors'] = 100
    umap_kwargs['min_dist'] = 0.25
    # umap_kwargs['spread'] = 1.0
    # umap_kwargs['metric'] = 'euclidean'

    # Step 2) make/load data
    # ...

    #smod = ''
    smod = '_last'

    agg_dir = manyruns_path + os.sep + 'aggregate'
    fpath_state = agg_dir + os.sep + 'X_aggregate%s.npz' % smod
    fpath_energy = agg_dir + os.sep + 'X_energy%s.npz' % smod
    fpath_pickle = manyruns_path + os.sep + 'multicell_template.pkl'

    X = np.load(fpath_state)['arr_0'].T  # umap wants transpose
    X_energies = np.load(fpath_energy)['arr_0'].T  # umap wants transpose (?)
    with open(fpath_pickle, 'rb') as pickle_file:
        multicell_template = pickle.load(pickle_file)  # unpickling multicell object

    if use_01:
        X = (1 + X) / 2.0
        X = X.astype(int)

    print('accessing', j, manyruns_path)
    X_multi[j, :, :] = X[0:nn, 0:kk]

In [ ]:
# UMAP aligned needs a relationdict for the 'time varying' dataset
# our relation is that each traj maps to itself (in time) -- constant relation

constant_dict = {i: i for i in range(kk)}
constant_relations = [constant_dict for i in range(len(gamma_list)-1)]

In [ ]:
#X_multi_as_list = [X_multi[i,:,:] for i in range(X_multi.shape[0])]
aligned_mapper = umap.AlignedUMAP().fit(X_multi, relations=constant_relations)

In [ ]:
def axis_bounds(embedding):
    left, right = embedding.T[0].min(), embedding.T[0].max()
    bottom, top = embedding.T[1].min(), embedding.T[1].max()
    adj_h, adj_v = (right - left) * 0.1, (top - bottom) * 0.1
    return [left - adj_h, right + adj_h, bottom - adj_v, top + adj_v]

In [ ]:
num_rows = 4
num_cols = 4
fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 40))
ax_bound = axis_bounds(np.vstack(aligned_mapper.embeddings_))
for i, ax in enumerate(axs.flatten()):
    if i<len(gamma_list):
        #current_target = ordered_target[150 * i:min(ordered_target.shape[0], 150 * i + 400)]
        
        manyruns_path = manyruns_paths[i]
        print(i, gamma_list[i], manyruns_path)
        fpath_energy = manyruns_path + os.sep + 'aggregate' + os.sep + 'X_energy_last.npz'
        X_energies = np.load(fpath_energy)['arr_0'].T  # umap wants transpose (?)
        energies = X_energies[:nn, 0]
        
        #ax.scatter(*aligned_mapper.embeddings_[i].T, s=2, cmap="Spectral_r")
        ax.scatter(*aligned_mapper.embeddings_[i].T, s=10, c=energies, cmap="Spectral_r")
        
        ax.axis(ax_bound)
        ax.set(xticks=[], yticks=[])
plt.tight_layout()
plt.savefig('aligned%d_gammas%d.jpg' % (nn, len(gamma_list)), dpi=300)

In [ ]:
picks = [14, 11, 10, 9, 7, 6, 5, 0]
picks = [0, 1]

num_rows = 1
num_cols = len(picks)

fig, axs = plt.subplots(num_rows, num_cols, figsize=(30, 10))
ax_bound = axis_bounds(np.vstack(aligned_mapper.embeddings_))
for j, ax in enumerate(axs.flatten()):
    if j<len(picks):
        
        i = picks[j]
        #current_target = ordered_target[150 * i:min(ordered_target.shape[0], 150 * i + 400)]
        
        manyruns_path = manyruns_paths[i]
        print(i, gamma_list[i], manyruns_path)
        fpath_energy = manyruns_path + os.sep + 'aggregate' + os.sep + 'X_energy_last.npz'
        X_energies = np.load(fpath_energy)['arr_0'].T  # umap wants transpose (?)
        energies = X_energies[:nn, 0]
        
        #ax.scatter(*aligned_mapper.embeddings_[i].T, s=2, cmap="Spectral_r")
        ax.set_title(gamma_list[i], fontsize=20)
        sc = ax.scatter(*aligned_mapper.embeddings_[i].T, s=10, c=energies, cmap="Spectral_r")
        
        if i == 1:
            cbar = plt.colorbar(sc)
            cbar.ax.tick_params(size=0)

        ax.axis(ax_bound)
        ax.set(xticks=[], yticks=[])
plt.tight_layout()
plt.savefig('picks_aligned%d_gammas%d.jpg' % (nn, len(gamma_list)), dpi=300)

In [ ]:
#picks = [14, 11, 10, 9, 7, 6, 5, 1]
#picks = [13, 11, 10, 9, 7, 6, 2, 1]
picks = [13, 9, 7, 6, 4, 3, 1, 0]
#picks = [13, 7, 6, 5, 4, 3, 1, 0]
#picks = [15, 14, 13, 7, 6, 5, 4, 3, 2, 1, 0]
#picks = [14, 11, 10, 9, 7, 6, 5, 0]
#picks = np.arange(len(gamma_list))

print("Max picks:", len(manyruns_paths) - 1)
print("Picks:", [gamma_list[p] for p in picks])


square = True

num_rows = 1
num_cols = len(picks)

if square:
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(30, 7))  # SQUARE 30 x 5
else:
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(30, 10))  # TALL 30 x 10

ax_bound = axis_bounds(np.vstack(aligned_mapper.embeddings_))
for j, ax in enumerate(axs.flatten()):
    if j<len(picks):
        
        i = picks[j]
        #current_target = ordered_target[150 * i:min(ordered_target.shape[0], 150 * i + 400)]
        
        manyruns_path = manyruns_paths[i]
        print(i, gamma_list[i], manyruns_path)
        fpath_energy = manyruns_path + os.sep + 'aggregate' + os.sep + 'X_energy_last.npz'
        X_energies = np.load(fpath_energy)['arr_0'].T  # umap wants transpose (?)
        energies = X_energies[:nn, 0]
        
        #ax.scatter(*aligned_mapper.embeddings_[i].T, s=2, cmap="Spectral_r")
        #ax.set_title(gamma_list[i], fontsize=30, y=None)
        if square:
            ax.set_title(gamma_list[i], fontsize=30, y=-0.12)
            sc = ax.scatter(*aligned_mapper.embeddings_[i].T, s=5, c=energies, cmap="Spectral_r")
        else:
            ax.set_title(gamma_list[i], fontsize=30, y=-0.06)
            sc = ax.scatter(*aligned_mapper.embeddings_[i].T, s=10, c=energies, cmap="Spectral_r")

        
        #cbar = plt.colorbar(sc)
        #cbar.ax.tick_params(size=0)

        #ax.axis(ax_bound)
        ax.set(xticks=[], yticks=[])
plt.tight_layout()
plt.savefig('picks_nobound_aligned%d_gammas%d.jpg' % (nn, len(gamma_list)), dpi=300)

In [ ]:
#picks = [14, 11, 10, 9, 7, 6, 5, 1]
#picks = [14, 11, 10, 9, 7, 6, 5, 1]
#picks = [14, 11, 10, 9, 7, 6, 5, 0]
#picks = [0]

#picks = [15, 13, 7, 5, 4, 2, 1, 0]
#picks = [13, 12, 11, 10, 9, 8, 7, 0]#, 4, 3, 1, 0]
picks = [15, 13, 8, 5, 3, 2, 1, 0]
picks = picks[::-1]

square = True

num_rows = 2
num_cols = 4

if square:
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(18*0.68, 12*0.68))  # SQUARE 30 x 5
else:
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(30, 10))  # TALL 30 x 10

ax_bound = axis_bounds(np.vstack(aligned_mapper.embeddings_))
for j, ax in enumerate(axs.flatten()):
    if j<len(picks):
        
        i = picks[j]
        #current_target = ordered_target[150 * i:min(ordered_target.shape[0], 150 * i + 400)]
        
        manyruns_path = manyruns_paths[i]
        print(i, gamma_list[i], manyruns_path)
        fpath_energy = manyruns_path + os.sep + 'aggregate' + os.sep + 'X_energy_last.npz'
        X_energies = np.load(fpath_energy)['arr_0'].T  # umap wants transpose (?)
        energies = X_energies[:nn, 0]
        
        #ax.scatter(*aligned_mapper.embeddings_[i].T, s=2, cmap="Spectral_r")
        #ax.set_title(gamma_list[i], fontsize=30, y=None)
        if square:
            ax.set_title(gamma_list[i], fontsize=20, y=-0.12)
            sc = ax.scatter(*aligned_mapper.embeddings_[i].T, s=2, c=energies, cmap="Spectral_r")
        else:
            ax.set_title(gamma_list[i], fontsize=30, y=-0.06)
            sc = ax.scatter(*aligned_mapper.embeddings_[i].T, s=10, c=energies, cmap="Spectral_r")

        
        #cbar = plt.colorbar(sc)
        #cbar.ax.tick_params(size=0)

        #ax.axis(ax_bound)
        ax.set(xticks=[], yticks=[])
plt.tight_layout()
plt.savefig('picks_nobound_2x4_aligned%d_gammas%d.jpg' % (nn, len(gamma_list)), dpi=300)